# 飞桨常规赛：遥感影像地块分割 - 9月第3名方案

本次的飞桨常规赛主要是对遥感影像进行像素级内容解析，并对遥感影像中感兴趣的类别进行提取和分类，以衡量遥感影像地块分割模型在多个类别（如建筑、道路、林地等）上的效果。

我采用的方案是deeplabv3+, 并使用了diceloss替换了原先的celoss, 在GPU的环境下训练了8个小时左右得到的50.316的精度, 整体的训练和测试流程如下

## 数据处理和paddleseg框架安装

模型训练之前我们需要将将数据进行解压, 并生成相应的训练集和验证集的txt文件。


In [ ]:
# 解压数据
%cd /home/aistudio/data/data80164/
!unzip -oq /home/aistudio/data/data80164/train_and_label.zip
!unzip -oq /home/aistudio/data/data80164/img_test.zip

/home/aistudio
/home/aistudio/tmp_data


In [ ]:
# 安装paddleseg框架
%cd /home/aistudio/work
!git clone https://gitee.com/paddlepaddle/PaddleSeg.git


/home/aistudio/work
Cloning into 'PaddleSeg'...
remote: Enumerating objects: 2532, done.
remote: Counting objects: 100% (2532/2532), done.
remote: Compressing objects: 100% (1400/1400), done.
remote: Total 14316 (delta 1403), reused 2166 (delta 1102), pack-reused 11784
Receiving objects: 100% (14316/14316), 337.68 MiB | 24.50 MiB/s, done.
Resolving deltas: 100% (9235/9235), done.
Checking connectivity... done.


In [ ]:
# 安装并测试paddleseg
%cd PaddleSeg
!pip install paddleseg
!python predict.py --config configs/quick_start/bisenet_optic_disc_512x512_1k.yml --model_path https://bj.bcebos.com/paddleseg/dygraph/optic_disc/bisenet_optic_disc_512x512_1k/model.pdparams --image_path docs/images/optic_test_image.jpg --save_dir output/result

/home/aistudio/work/PaddleSeg
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 245kB 10.5MB/s eta 0:00:01
Connecting to https://paddleseg.bj.bcebos.com/dataset/optic_disc_seg.zip
[==================================================] 100.00%
Uncompress optic_disc_seg.zip
[==================================================] 100.00%
2021-10-16 10:37:14 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 1000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  pretrained: null
  type: BiSeNetV2
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: data/optic_disc_seg
  mode: train
  transforms:
  - target_size:
    - 512
    - 512
    type: Resize
  - type: RandomHorizontalFlip
  - type: Normalize
  type: OpticDiscSeg
val_dataset:
  dataset_root: data/opt

In [ ]:
# 分割数据集, 将数据集安装7: 3的比例分成训练集和验证集, 帮助后面我们在训练过程中保存表现最好的模型
%cd /home/aistudio/data/data80164/
!python tools/split_dataset_list.py /home/aistudio/tmp_data/ img_train lab_train --label_class '0' '1' '2' '3'  --format jpg png

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 模型训练

现在我们已经准备好了数据集和训练语义分割模型所要使用的框架和paddleseg

我们使用deeplabv3p作为我们的基本方法，主干特征提取网络选择为resnet50vd，损失函数则使用diceloss，相对于celoss而言对分割的表达能力更强，可以计算全局的分割相似度。借助paddleseg框架，
我们只需要在PaddleSeg的`configs/deeplabv3p/`目录下建立一个`deeplabv3p_remote.yml`的配置文件即可, 配置文件的内容如下：

```yaml
batch_size: 4
iters: 80000


train_dataset: #训练数据设置
  type: Dataset #选择数据集格式
  dataset_root: /home/aistudio/data/data80164/ # 选择数据集路径
  train_path: /home/aistudio/data/data80164/train.txt
  num_classes: 4 #指定目标的类别个数（背景也算为一类）
  transforms: #数据预处理/增强的方式
    - type: Resize #送入网络之前需要进行resize
      target_size: [256, 256] #将原图resize成512*512再送入网络
    - type: RandomHorizontalFlip #采用水平反转的方式进行数据增强
    - type: Normalize #图像进行归一化
  mode: train

val_dataset: #验证数据设置
  type: Dataset #选择数据集格式
  dataset_root: /home/aistudio/data/data80164/ #选择数据集路径
  val_path: /home/aistudio/data/data80164/val.txt
  num_classes: 4 #指定目标的类别个数（背景也算为一类）
  transforms: #数据预处理/增强的方式
    - type: Resize  #将原图resize成512*512在送入网络
      target_size: [256, 256]  #将原图resize成512*512在送入网络
    - type: Normalize #图像进行归一化
  mode: val


optimizer:
  type: sgd
  momentum: 0.9
  weight_decay: 4.0e-5

lr_scheduler:
  type: PolynomialDecay
  learning_rate: 0.01
  end_lr: 0
  power: 0.9

loss:
  types:
    - type: DiceLoss
  coef: [1]

model:
  type: DeepLabV3P
  backbone:
    type: ResNet50_vd
    output_stride: 8
    multi_grid: [1, 2, 4]
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/resnet50_vd_ssld_v2.tar.gz
  backbone_indices: [0, 3]
  aspp_ratios: [1, 12, 24, 36]
  aspp_out_channels: 256
  align_corners: True
  pretrained:None
```
完成之后，直接执行下面的脚本完成训练过程即可，其中batchsize大家可以根据自己的机器型号来进行调整。
训练完成之后的模型将会保存在`output/models_remote/deeplabv3p`目录下

In [ ]:
# 运行代码
%cd /home/aistudio/work/PaddleSeg
!python train.py --config configs/deeplabv3p/deeplabv3p_remote.yml --do_eval --use_vdl --save_interval 1000 --save_dir output/models_remote/deeplabv3p


/home/aistudio/work/PaddleSeg
2021-10-16 11:06:53 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-150-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Cuda compilation tools, release 10.1, V10.1.243
cudnn: 7.6
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-32GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.1.2
OpenCV: 4.1.1
------------------------------------------------
2021-10-16 11:06:53 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 80000
loss:
  coef:
  - 1
  types:
  - ignore_index: 255
    type: DiceLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  align_corners: true
  aspp_out_channels: 256
  aspp_ratios:
  - 1
  - 12
  - 24
  - 36
  backbone:
    multi_grid:
    - 1
    - 2
    - 4
    output_stride: 8
    pretrained: https://bj.bcebos.com/p

## 生成测试集结果

模型的训练过程比较漫长，算力卡不足的同学，可以使用中断，在配置文件中设置pretrained字段，从中断开始训练即可。

训练好的模型将会被保存在`work/PaddleSeg/output/models_remote/deeplabv3p/best_model`目录下，生成测试结果时，需要和官方要求的颜色保持一致，我们要将`PaddleSeg/paddleseg/utils/visualize.py`中第90行修改为
`color_map = [0,0,0, 1,1,1, 2,2,2, 3,3,3]`，之后执行下面验证的流程即可。


In [31]:
%cd /home/aistudio/work/PaddleSeg
!python predict.py --config configs/deeplabv3p/deeplabv3p_remote.yml --model_path output/models_remote/deeplabv3p/best_model/model.pdparams --image_path /home/aistudio/data/data80164/img_testA --save_dir output/result_remote

/home/aistudio/work/PaddleSeg
2021-10-16 11:56:05 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 80000
loss:
  coef:
  - 1
  types:
  - type: DiceLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  align_corners: true
  aspp_out_channels: 256
  aspp_ratios:
  - 1
  - 12
  - 24
  - 36
  backbone:
    multi_grid:
    - 1
    - 2
    - 4
    output_stride: 8
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/resnet50_vd_ssld_v2.tar.gz
    type: ResNet50_vd
  backbone_indices:
  - 0
  - 3
  type: DeepLabV3P
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: /home/aistudio/data/data80164/
  mode: train
  num_classes: 4
  train_path: /home/aistudio/data/data80164/train.txt
  transforms:
  - target_size:
    - 256
    - 256
    type: Resize
  - type: RandomHorizontalFlip
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: /home/aistudio/data/data80164/
  m

## 提交结果
结果将会保存在`work/PaddleSeg/output/result_remote/added_prediction` 目录下，下载之后提交即可

本次常规赛主要是借助了paddleseg框架完成了这次语义分割任务，通过deeplabv3p+diceloss的形式就可以拿到不错的成绩，后面还需要继续学习相关知识，可以利用后面的机会在ocrnet等更高级的语义分割网络上做做实验，还有在优化器和损失函数上也有很多需要改进的地方。

最后，感觉paddle提供了这么方便的平台，提供的V100非常给力，paddle赞！